In [3]:
from pathlib import Path
import os
import torch
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import datetime

In [4]:
from datamaestro import prepare_dataset
ds = prepare_dataset("com.lecun.mnist");
train_images, train_labels = ds.train.images.data(), ds.train.labels.data()
test_images, test_labels =  ds.test.images.data(), ds.test.labels.data()

## Question 1 

Implémenter un dataset pour le jeu de données MNIST qui renvoie une image
sous la forme d’un vecteur normalisé entre 0 et 1, et le label associé (sans utiliser
TensorDataset). Tester votre dataset avec un dataloader pour différentes tailles de
batch et explorer les options du dataloader.
Vous pouvez vous référer à la doc officielle.

In [5]:
#verfier le nombre d'image dans le train max -> (60000, 28, 28)
# idem pour test -> (10000, 28, 28)
train_images = train_images/train_images.max()
test_images = test_images/test_images.max()
batch = 100


In [6]:
# on s'assure que les images sont normalisés
print(train_images.min(),train_images.max(),test_images.min(),test_images.max())

0.0 1.0 0.0 1.0


In [8]:
from torch.utils.data import TensorDataset

#Pour utiliser Tensor il faut que ca soit des tensors... 
train_labels = torch.from_numpy(train_labels)
train_images = torch.from_numpy(train_images)
train_dataset = TensorDataset(train_images, train_labels)

test_labels = torch.from_numpy(test_labels)
test_images = torch.from_numpy(test_images)
test_dataset = TensorDataset(test_images, test_labels)

C:\Users\enza6\AppData\Local\Temp\ipykernel_26472\2399840432.py:4: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:212.)
  train_labels = torch.from_numpy(train_labels)


In [9]:
train_data = DataLoader(dataset=train_dataset, batch_size = batch, shuffle = True)
test_data = DataLoader(dataset=test_dataset, batch_size = batch, shuffle = True)

## Gestion mémoire

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else ('cpu'))
device

device(type='cpu')

## Checkpointing
Créer une classe pour stocker l'état du modèle et de l'optimiseur lors de l'entraînement.

In [11]:
class State : 
    def __init__(self, model, optim) -> None:
        self.model = model 
        self.optim = optim
        self.epoch, self.iteration = 0, 0

Dans les itérations d'entraînement : 
    Réinitialise les gradients de l'optimiseur avec state.optim.zero_grad().
    Déplace les données d'entrée x vers le périphérique spécifié (device).
    Passe les données d'entrée x dans le modèle state.model pour obtenir les prédictions xhat.
    Calcule la perte (loss) entre les prédictions xhat et les données d'entrée x.
    Effectue la rétropropagation (backpropagation) de la perte pour calculer les gradients.
    Applique une étape d'optimisation avec state.optim.step() pour mettre à jour les poids du modèle.
    Incrémente le compteur d'itérations state.iteration

A la fin de chaque époque, il sauvegarde l'état courant de l'entrainement dans un fichier créer au debut en utilisant 'torch.save'.

## Implémentation d'un autoencodeur
Implémenter une classe autoencodeur (héritée de Module) selon l’architecture sui-
vante : linéaire → Relu pour la partie encodage et linéaire → sigmoïde pour la partie
décodage. 

Les poids du décodeur correspondent usuellement à la transposée des poids
de l’encodeur (quel est l’avantage ?)

[A répondre]

In [12]:
class Autoencoder(nn.Module): 
    def __init__(self):
        #beginning size image = 784
        super(Autoencoder, self).__init__() 
        self.encoder = nn.Sequential(
            nn.Linear(test_images.shape[1]*test_images.shape[2],128), # 784->128
            nn.ReLU(), 
            nn.Linear(128,64),
            nn.ReLU(), 
            nn.Linear(64,12),
            nn.ReLU(), 
            nn.Linear(12,3)# N -> 3 
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(3,12),
            nn.Sigmoid(), 
            nn.Linear(12,64),
            nn.Sigmoid(), 
            nn.Linear(64,128),
            nn.Sigmoid(), 
            nn.Linear(128,test_images.shape[1]*test_images.shape[2]),
            ## need a active fonction wich but value between 0 and 1
            nn.Sigmoid()
        )
        pass
    
    def forward (self, x): 
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [23]:
model = Autoencoder()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-5)

In [24]:
log_dir = "log"
writer = SummaryWriter(log_dir)

In [25]:
#training
num_epochs = 10
outputs    = []
for epoch in range(num_epochs): 
    LOSS = []
    for (img, _) in train_data: 
        #on prend les images une par une
        img   = img.reshape(-1, train_images.shape[1]*train_images.shape[2])
        #reconstructed image 
        img = img.float()
        recon = model(img)
        loss  = criterion(recon, img)
        
        writter.add_scalar("Loss",loss, epoch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        LOSS.append(loss.item())
        outputs.append((epoch,img,recon))
    print(f'Epoch:{epoch+1}/{num_epochs},Loss:{sum(LOSS)/len(LOSS)}')
writer.close()

Epoch:1/10,Loss:0.20252125054597855
Epoch:2/10,Loss:0.14667908685902756
Epoch:3/10,Loss:0.10919386795411508
Epoch:4/10,Loss:0.08875401835888624
Epoch:5/10,Loss:0.07861191418021918
Epoch:6/10,Loss:0.07362142692009609
Epoch:7/10,Loss:0.07103535457203786
Epoch:8/10,Loss:0.06960473871479432
Epoch:9/10,Loss:0.0687680820375681
Epoch:10/10,Loss:0.06825562366594871


In [26]:
#test
model.eval()
outputs    = []
test_loss = 0.0
for (img, _) in test_data: 
    #on prend les images une par une
    img   = img.reshape(-1, train_images.shape[1]*train_images.shape[2])
    #reconstructed image 
    img = img.float()
    recon = model(img)
    loss  = criterion(recon, img)
       
    writter.add_scalar("Loss",loss)
        
    test_loss += loss.item()
            
average_loss = test_loss/len(test_labels)
print(f'Average Test Loss : {average_loss:.4f}')

Average Test Loss : 0.0007


In [27]:
#%load_ext tensorboard
#%tensorboard --logdir logs